In [1]:
import tensorflow as tf
import numpy as np
import datetime, time

In [10]:
def genObs(length, dict_size=10):
    base = np.random.choice(range(dict_size), size=length)
    return base

# start and finish of lengthiest ascending sequence
def getTarget(x):
    prev = x[0]
    start = 0
    cs = 0
    ce = 0
    cl = 0
    
    for i in range(1, len(x)):
        if prev > x[i]:
            if i - start > cl:
                cl = i - start
                cs = start
                ce = i - 1
            start = i
        prev = x[i]
    
    if len(x) - start > cl:
        cs = start
        ce = len(x) - 1
    return (x[cs], x[ce])

def genSample(num, length=20):
    x = [genObs(length) for _ in range(num)]
    y = [getTarget(y) for y in x]
    return np.array(x), np.array(y)

def randomBatch(tensorTuple, batchSize=64):
    ids = np.random.choice(range(tensorTuple[0].shape[0]), batchSize)
    return (x[ids,] for x in tensorTuple)

def shuffleBatches(tensorTuple, batchSize=64):
    if type(tensorTuple) is list or type(tensorTuple) is tuple: 
        ids = list(range(tensorTuple[0].shape[0]))
        np.random.shuffle(ids)
        for i in range(0,len(ids),batchSize):
            lst = min(len(ids), i + batchSize)
            yield (np.array(x[ids[i:lst],]) for x in tensorTuple)
    else:
        ids = list(range(tensorTuple.shape[0]))
        np.random.shuffle(ids)
        for i in range(0,len(ids),batchSize):
            lst = min(len(ids), i + batchSize)
            yield np.array(tensorTuple[ids[i:lst],])

In [12]:
train_x, train_y = genSample(100000)
valid_x, valid_y = genSample(10000)
valid_x[:10], valid_y[:10]

(array([[3, 8, 1, 9, 1, 8, 7, 3, 3, 2, 0, 9, 0, 4, 7, 0, 3, 2, 6, 1],
        [8, 6, 5, 9, 9, 5, 1, 5, 1, 0, 4, 4, 1, 2, 2, 9, 5, 8, 5, 3],
        [0, 3, 6, 1, 4, 8, 8, 3, 9, 6, 1, 3, 4, 9, 8, 0, 1, 1, 3, 1],
        [0, 9, 7, 1, 3, 4, 0, 0, 9, 0, 4, 9, 5, 4, 5, 6, 8, 6, 4, 0],
        [9, 2, 9, 7, 1, 6, 4, 3, 1, 1, 1, 1, 7, 0, 0, 2, 3, 0, 3, 7],
        [3, 0, 9, 9, 4, 0, 3, 5, 7, 9, 7, 5, 0, 4, 7, 4, 9, 6, 8, 7],
        [8, 7, 5, 5, 9, 8, 4, 5, 0, 1, 4, 0, 8, 5, 3, 7, 8, 8, 4, 0],
        [0, 3, 8, 3, 2, 9, 6, 1, 8, 3, 7, 6, 2, 0, 9, 7, 7, 4, 3, 2],
        [0, 1, 6, 3, 0, 6, 4, 5, 2, 1, 5, 5, 1, 9, 6, 6, 4, 5, 1, 2],
        [1, 0, 2, 1, 8, 2, 0, 3, 8, 9, 6, 7, 3, 4, 3, 6, 0, 2, 1, 2]]),
 array([[0, 7],
        [1, 9],
        [1, 8],
        [4, 8],
        [1, 7],
        [0, 9],
        [3, 8],
        [0, 8],
        [0, 6],
        [0, 9]]))

In [14]:
DICT_SIZE = 10
RNN_SIZE = [12]
HIDDEN_LAYER = 30

RCell = lambda n: tf.nn.rnn_cell.GRUCell(num_units=n, activation=tf.nn.elu)

tf.reset_default_graph()

rnnCell = tf.nn.rnn_cell.MultiRNNCell([RCell(s) for s in RNN_SIZE], state_is_tuple=True)

tfi_x = tf.placeholder(shape=(None, None), dtype=tf.int32)
tfi_y = tf.placeholder(shape=(None, 2), dtype=tf.int32)

tfX = tf.one_hot(tfi_x, DICT_SIZE, dtype=tf.float32)
tfY = tf.one_hot(tfi_y, DICT_SIZE, dtype=tf.float32)

_, tfFin0 = tf.nn.dynamic_rnn(rnnCell, inputs=tfX, dtype=tf.float32, time_major=False)
tfFin = tfFin0[-1] #get latest layer in RNN

tfRes0 = tf.layers.dense(tfFin, HIDDEN_LAYER, activation=tf.nn.elu)
tfRes1 = tf.layers.dense(tfRes0, DICT_SIZE * 2)

tfRes = tf.reshape(tfRes1, shape=(tf.shape(tfRes1)[0], 2, DICT_SIZE))

tfLoss0 = tf.nn.softmax_cross_entropy_with_logits(labels=tfY, logits=tfRes) #should be like batch_size x SEQ_LEN-1
tfLoss = tf.reduce_mean(tfLoss0)
tfTrain = tf.train.AdamOptimizer(1e-3).minimize(tfLoss)

tfPredicted = tf.argmax(tfRes, axis=2)
tfActual = tf.argmax(tfY, axis=2)

tfAccuracy = tf.reduce_mean(tf.cast(tf.equal(tfActual, tfPredicted), dtype=tf.float32))

#tfsLoss = tf.summary.scalar('RMSE', tfLoss)
#tfsAccuracy = tf.summary.scalar('Accuracy', 1-tfAccuracyObs)
#tfsAll = tf.summary.merge([tfsLoss, tfsAccuracy])
tfsSaver = tf.train.Saver()

tffw = tf.summary.FileWriter('D:/Jupyter/Logs/00_A', tf.get_default_graph())
print('Graph creation complete')

Graph creation complete


In [16]:
dt_now = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
#tffw = tf.summary.FileWriter('D:/Jupyter/Logs/12RNN07-{0}'.format(dt_now), tf.get_default_graph())

batch_size = 1000
num_steps  = 10
num_epochs = 100
checkpoints = 5 #every 5 epochs save the model

fmtstr = 'Epoch {0} ({1:1.3} sec): \t\tVL:{2:1.3f}\t\tAC:{3:1.3f}'
valid_batch = {tfi_x: valid_x, tfi_y: valid_y}
with tf.Session() as tfs:
    tfs.run(tf.global_variables_initializer())
    for i in range(num_epochs):
        te0 = time.perf_counter()
        for (mini_x, mini_y) in shuffleBatches((train_x, train_y), batchSize=batch_size):
            train_batch = {tfi_x:mini_x, tfi_y: mini_y}
            
            #l0 = tfLoss.eval(feed_dict=train_batch)
            #t0 = time.perf_counter()
            for j in range(num_steps):
                tfTrain.run(feed_dict=train_batch)
            #t1 = time.perf_counter()
            l1 = tfLoss.eval(feed_dict=train_batch)
    
        te1 = time.perf_counter()
        [lv,ac] = tfs.run([tfLoss, tfAccuracy], feed_dict=valid_batch)
            #tffw.add_summary(summary, i)
        if i%checkpoints == 0 and i > 0:
            p = tfsSaver.save(tfs, 'D:/Jupyter/mltest/Models-13RNN02/model-{0:02d}.ckpt'.format(i))
            print('Model saved at checkpoint: {0}'.format(p))
                             
        print(fmtstr.format(i,te1-te0,lv,ac))
    valid_r = tfs.run(tfOutR, feed_dict=valid_batch)

Epoch 0 (33.1 sec): 		VL:1.440		AC:0.479
Epoch 1 (32.9 sec): 		VL:1.305		AC:0.532
Epoch 2 (33.0 sec): 		VL:1.180		AC:0.586
Epoch 3 (32.4 sec): 		VL:1.088		AC:0.620
Epoch 4 (32.5 sec): 		VL:0.981		AC:0.664
Model saved at checkpoint: D:/Jupyter/mltest/Models-13RNN02/model-05.ckpt
Epoch 5 (33.8 sec): 		VL:0.910		AC:0.699
Epoch 6 (34.6 sec): 		VL:0.852		AC:0.721
Epoch 7 (33.6 sec): 		VL:0.797		AC:0.739
Epoch 8 (32.2 sec): 		VL:0.741		AC:0.760
Epoch 9 (31.0 sec): 		VL:0.675		AC:0.785
Model saved at checkpoint: D:/Jupyter/mltest/Models-13RNN02/model-10.ckpt
Epoch 10 (31.0 sec): 		VL:0.605		AC:0.807
Epoch 11 (31.1 sec): 		VL:0.541		AC:0.829
Epoch 12 (31.0 sec): 		VL:0.481		AC:0.847
Epoch 13 (31.3 sec): 		VL:0.430		AC:0.864
Epoch 14 (31.7 sec): 		VL:0.383		AC:0.881
Model saved at checkpoint: D:/Jupyter/mltest/Models-13RNN02/model-15.ckpt
Epoch 15 (31.1 sec): 		VL:0.332		AC:0.900
Epoch 16 (31.1 sec): 		VL:0.308		AC:0.911
Epoch 17 (31.0 sec): 		VL:0.283		AC:0.914


KeyboardInterrupt: 